Project Overview


Objective: Build a system that generates relevant questions from given text passages using the T5 model.

Dataset

SQuAD (Stanford Question Answering Dataset): Contains passages and corresponding questions and answers.

I  choosed this dataset cause it's very common for this kind of task.

In [11]:
from datasets import load_dataset

dataset = load_dataset('squad')


Data Preparation

In [12]:
def prepare_data(example):
    example['source_text'] = "generate question: " + example['context']
    example['target_text'] = example['question']
    return example

dataset = dataset.map(prepare_data)


Tokenization

In [18]:
def tokenize(batch):
    from transformers import T5Tokenizer  # Import inside the function
    tokenizer = T5Tokenizer.from_pretrained('t5-small')
    
    source = tokenizer(
        batch['source_text'],
        padding='max_length',
        truncation=True,
        max_length=512
    )
    target = tokenizer(
        batch['target_text'],
        padding='max_length',
        truncation=True,
        max_length=64
    )

    batch['input_ids'] = source['input_ids']
    batch['attention_mask'] = source['attention_mask']
    batch['labels'] = target['input_ids']
    return batch


# Tokenize the dataset
tokenized_dataset = dataset.map(
    tokenize,
    batched=True,
    batch_size=1000,       
    num_proc=4,           
    remove_columns=dataset['train'].column_names,
)



Map (num_proc=4): 100%|██████████| 10570/10570 [00:08<00:00, 1241.83 examples/s]


Fine-Tuning the T5 Model

In [19]:
# Proceed with model initialization and training
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer

model = T5ForConditionalGeneration.from_pretrained('t5-small')

In [20]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy='epoch',
    save_steps=10_000,
    save_total_limit=2,
)


c:\Users\Hesan\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
)

In [22]:
trainer.train()


  1%|          | 500/65700 [19:02<41:43:09,  2.30s/it]

{'loss': 1.0469, 'grad_norm': 1.2295717000961304, 'learning_rate': 4.961948249619483e-05, 'epoch': 0.02}


  2%|▏         | 1000/65700 [38:51<43:52:42,  2.44s/it]

{'loss': 0.5507, 'grad_norm': 1.3147540092468262, 'learning_rate': 4.923896499238965e-05, 'epoch': 0.05}


  2%|▏         | 1500/65700 [58:45<42:50:44,  2.40s/it]

...

{'loss': 0.47, 'grad_norm': 0.9896535277366638, 'learning_rate': 3.439878234398782e-05, 'epoch': 0.94}


 32%|███▏      | 21000/65700 [23:30:49<65:08:32,  5.25s/it]

{'loss': 0.4711, 'grad_norm': 1.4654635190963745, 'learning_rate': 3.4018264840182654e-05, 'epoch': 0.96}


 32%|███▏      | 21236/65700 [23:47:51<27:57:05,  2.26s/it] 

KeyboardInterrupt: 

In [ ]:
# Save the trained model
trainer.save_model('./trained_t5_question_generator')